In [8]:
from IPython.display import display
from PIL import Image
import cv2
import numpy as np

def imshow(image):
    display(Image.fromarray(image))

In [9]:
board = cv2.imread("data/board.jpg")

In [10]:
def get_mask_of_board(frame_bgr):
    hsv = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2HSV)
    
    mask = cv2.inRange(hsv, np.array([20, 40, 40]), np.array([140, 255, 255]))
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    filtered = [c for c in contours if cv2.contourArea(c) > 500]
    
    all_points = np.vstack(filtered)
    hull = cv2.convexHull(all_points)
    
    board_mask = np.zeros_like(mask)
    cv2.drawContours(board_mask, [hull], -1, 255, -1)
    
    return board_mask

board_mask = get_mask_of_board(board)

board = cv2.bitwise_and(board, board, mask=board_mask)
boardimage = cv2.cvtColor(board, cv2.COLOR_BGR2RGB)

In [11]:
def preprocess(image):
    # gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.GaussianBlur(image, (5, 5), 0)
    return image

In [12]:
board = preprocess(board)

In [13]:
orb = cv2.ORB_create(
    nfeatures=5000,
    scaleFactor=1.2,
    nlevels=8
)

kp_board, des_board = orb.detectAndCompute(board, None)

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)

In [15]:
cap = cv2.VideoCapture("data/pro1.mp4")

fps = cap.get(cv2.CAP_PROP_FPS)
delay_ms = int(1000 / fps) if fps and fps > 0 else 30

while True:
    ret, frame = cap.read()
    frame = preprocess(frame)
    if not ret:
        break
    
    kp_frame, des_frame = orb.detectAndCompute(frame, None)
    matches = bf.knnMatch(des_board, des_frame, k=2)
    
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    src_pts = np.float32(
        [kp_board[m.queryIdx].pt for m in good_matches]
    ).reshape(-1, 1, 2)

    dst_pts = np.float32(
        [kp_frame[m.trainIdx].pt for m in good_matches]
    ).reshape(-1, 1, 2)

    match_img = cv2.drawMatches(
        frame, kp_frame,
        board, kp_board,
        good_matches[:25],
        None,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
    )


    H, mask = cv2.findHomography(
        src_pts,
        dst_pts,
        cv2.RANSAC,
        ransacReprojThreshold=5.0
    )

    if H is None:
        raise RuntimeError("Homography failed")

    h, w = board.shape[:2]
    warped = cv2.warpPerspective(frame, np.linalg.inv(H), (w, h))


    cv2.imshow("video", warped)

    if cv2.waitKey(delay_ms) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.12.0) /io/opencv/modules/features2d/src/draw.cpp:241: error: (-215:Assertion failed) i1 >= 0 && i1 < static_cast<int>(keypoints1.size()) in function 'drawMatches'
